In [1]:
# !brew install mongodb-community
# !mkdir -p /data/db
# !chown -R 'id -un'/data/db
# !brew services start mongodb-community

/bin/bash: brew: command not found
mkdir: /data: Read-only file system
usage: chown [-fhnvx] [-R [-H | -L | -P]] owner[:group] file ...
       chown [-fhnvx] [-R [-H | -L | -P]] :group file ...
/bin/bash: brew: command not found


In [4]:
import pymongo
from pymongo import MongoClient

In [480]:
import pandas as pd
import gspread
import selenium
import requests
from bs4 import BeautifulSoup
import time
import os
import openai

In [481]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [482]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By

In [6]:
client = MongoClient("localhost", 27017) 
B = client["collection"]

In [580]:
options = Options()
options.headless = True
driver = webdriver.Chrome('Bolic/chromedriver')

/var/folders/0p/m7h9nfyj06q0sft4l6k4r5jr0000gq/T/ipykernel_5046/2929111194.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('Bolic/chromedriver')


In [29]:
with open("env_creds.txt", "r") as f:
    l=f.read()
user_id=l.replace("\\\n", " ").split(" ")[-3:-1][0]
psswd=l.replace("\\\n", " ").split(" ")[-3:-1][1]

['random@wisc.edu', 'silly_password123']

In [581]:
driver.get("https://www.linkedin.com/")

In [582]:
email=driver.find_element(By.XPATH,"//input[@name='session_key']")
email.send_keys(user_id)
password=driver.find_element(By.XPATH,"//input[@name='session_password']")
password.send_keys(psswd)
time.sleep(3)
submit = driver.find_element(By.XPATH,"//button[@type='submit']").click()
time.sleep(3)

In [437]:
#personal bloggger function
def personal_blog_post(prompt):
    response=openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "LinkedIn Blog Post"},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

In [490]:

def message_post(prompt):
    response=openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "LinkedIn Message Request"},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

In [463]:
#any content
def blog_post (stext):
    response=openai.Completion.create(engine="text-davinci-003", prompt=stext,temperature=.1, max_tokens=200, top_p=1, frequency_penalty=1, presence_penalty=0)
    content = response.choices[0].text
    return content

In [578]:
#making posts:
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Start a post']")))
button.click()

In [500]:
#blank box
y=driver.find_element(By.CLASS_NAME, "ql-editor.ql-blank")
y.send_keys(blog_post("What kinds of cats are there"))

In [465]:
#post button
z= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Post']")))
z.click()

Automating LinkedIN Connections

In [547]:
#networking and connections
network_url = "https://www.linkedin.com/mynetwork/"  
driver.get(network_url)

In [ ]:
#making relevant connections

In [548]:
attempt_list=[]
see_elements = driver.find_elements(By.TAG_NAME, "button")
for i in range(len(see_elements)-1, -1,-1):
    if see_elements[i].text=="See all":
        see_elements[i].click  
        discover_person_cards=driver.find_elements(By.CLASS_NAME, 'app-aware-link')
        for i in range(len(discover_person_cards)):
            if i==len(discover_person_cards)-2:
                break
            elif i==0 or i%2==0:
                href = discover_person_cards[i].get_attribute('href')
                name = discover_person_cards[i].text
                attempt_list.append((name, href))

In [552]:
example_list=[item  for item in attempt_list if 'MEMBER_PROFILE_CANNED_SEARCH' not in item[1] ]
new_list=[]
from linkedin_api import Linkedin
application = Linkedin(user_id, psswd)
for item in example_list:
    if item[1].split("/")[-2]=="in":
        new_list.append(item)

In [573]:
industry_selec="Information Technology and Services"
school='University of Wisconsin-Madison'

In [575]:
for item in new_list:
    person_profile=application.get_profile(item[1].split("/")[-1])
    try:
        industry=person_profile['industryName']
        educ_list=[True for dicte in person_profile["education"] if school in dicte.values()]
    except:
        educ_list=[True for dicte in person_profile["education"] if school in dicte.values()]
    if True in educ_list or industry==industry_selec:
        print("nice")
        driver.get(item[1])
        time.sleep(2)
        button_elements = driver.find_elements(By.TAG_NAME, "button")
        try:
            button_elements[[item.text for item in button_elements].index("Connect")].click()
        except:
            continue
        time.sleep(3)
        new_button_elements = driver.find_elements(By.TAG_NAME, "button")
        new_button_elements[[item.text for item in new_button_elements].index("Add a note")].click()
        time.sleep(3)
        textarea_element = driver.find_element(By.ID, "custom-message")
        textarea_element.send_keys("Hey, I'm excited to connect!")
        time.sleep(3)
        new_new_button_elements = driver.find_elements(By.TAG_NAME, "button")
        new_button_elements[[item.text for item in new_new_button_elements].index("Send")].click()
        time.sleep(3)
        #break was implemented for testing purposes
        break
    else:
        new_list.remove(item)
        continue

nice
nice


This next feature was an exploratory search/application to easy apply jobs

In [535]:
search = driver.find_element(By.XPATH, "//input[@aria-label='Search']")
search.send_keys("Software Engineering Intern") #str(item[0]).split("\n")[1]
search.send_keys(Keys.RETURN)
time.sleep(2)
button_elements = driver.find_elements(By.TAG_NAME, "button")
button_elements[[item.text for item in button_elements].index("Jobs")].click()

In [576]:
button_elements = driver.find_elements(By.TAG_NAME, "button")
button_elements[[item.text for item in button_elements].index("Easy Apply")].click()

button_elements = driver.find_elements(By.TAG_NAME, "button")
button_elements[[button_elements[i].text for i in range(len(button_elements)-1, -1, -1)].index("Easy Apply")].click()

In [577]:
button_elements = driver.find_elements(By.TAG_NAME, "button")
button_elements[[item.text for item in button_elements].index("Next")].click()
button_elements = driver.find_elements(By.TAG_NAME, "button")
#resume selection
button_elements[[item.text for item in button_elements].index("Choose")].click()
input_elements = driver.find_elements(By.CLASS_NAME, "artdeco-text-input--input")

# send keys to each input element
for input_element in input_elements:
    input_element.send_keys("1")
    time.sleep(1)
    
button_elements = driver.find_elements(By.TAG_NAME, "button")
button_elements[[item.text for item in button_elements].index("Submit Application")].click()